In [1]:
import torchvision
import torch
from IPython.display import display
import ipywidgets
import traitlets
from robotpal import Camera, bgr8_to_jpeg, Robot
from robotpal.SCSCtrl import TTLServo
import torchvision.transforms as transforms
import cv2
import PIL.Image
import numpy as np
import os
import threading
import time
from datetime import datetime

# --------------------------------------------------------------
# 1. 초기 설정 (모델, 카메라, 로봇)
# --------------------------------------------------------------
# 모델 로드
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
# 파일명이 맞는지 확인하세요
model.load_state_dict(torch.load('best_steering_model_xy_test_12_17.pth', map_location=torch.device('cuda')))

device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ... ]

# 카메라 및 로봇 객체 생성 (해상도는 기본값 사용)
camera = Camera.instance()
robot = Robot()

[RobotPal] 서버 시작됨 | WebSocket: 9999, TCP: 9998
[RobotPal] 통신 대기 중... (WS: 9999, TCP: 9998)
[System] WebSocket 프로세서 시작


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


현재 모델 가중치는 CPU 메모리에 있습니다. 아래 코드를 실행하여 GPU 장치로 전송합니다.

In [ ]:
# --------------------------------------------------------------
# 2. [수정됨] 최적화된 색상 인식 스레드 (ROI + 갱신 제한)
# --------------------------------------------------------------

TTLServo.servoAngleCtrl(5, 25, 1, 100)

colorHSVvalueList = []
max_len = 20
th_flag = True  # 스레드 제어 플래그

# 위젯 초기화
colorlblMax = ipywidgets.Text(description="Max HSV:", value="Wait...")
colorlblMin = ipywidgets.Text(description="Min HSV:", value="Wait...")
image_widget = ipywidgets.Image(format='jpeg', width=300, height=300)

# 스냅샷용 변수
latest_annotated_frame = None
frame_lock = threading.Lock()

def colorRecog():
    global latest_annotated_frame, colorHSVvalueList
    
    # 화면 갱신 빈도를 조절할 카운터
    draw_counter = 0
    
    while th_flag:
        if camera.value is None:
            time.sleep(0.1)
            continue
            
        # 1. 이미지 복사 (필수)
        image = camera.value.copy()
        
        # 이미지 크기 확인 및 중심점 계산
        h, w, _ = image.shape
        cy, cx = int(h / 2), int(w / 2)
        
        # ---------------------------------------------------------
        # [최적화 1] 중앙 20x20 픽셀만 잘라서(ROI) 분석
        # 전체 이미지를 변환하지 않아 CPU 부하를 획기적으로 줄임
        # ---------------------------------------------------------
        roi_size = 10 # 중앙에서 +-10 (총 20x20 크기)
        
        # 범위 예외 처리
        y1 = max(0, cy - roi_size)
        y2 = min(h, cy + roi_size)
        x1 = max(0, cx - roi_size)
        x2 = min(w, cx + roi_size)
        
        # ROI 잘라내기 및 변환 (매우 빠름)
        roi = image[y1:y2, x1:x2]
        roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        roi_hsv = cv2.blur(roi_hsv, (5, 5)) # 작은 영역이라 블러도 빠름
        
        # ROI의 중심값 가져오기
        rh, rw, _ = roi_hsv.shape
        if rh > 0 and rw > 0:
            hsvValue = roi_hsv[int(rh/2), int(rw/2)]

            # 데이터 집계
            colorHSVvalueList.append(hsvValue)
            if len(colorHSVvalueList) > max_len:
                del colorHSVvalueList[0]
                
            if len(colorHSVvalueList) > 0:
                max_h, max_s, max_v = np.maximum.reduce(colorHSVvalueList)
                min_h, min_s, min_v = np.minimum.reduce(colorHSVvalueList)

                # 시각화 (원본 이미지 복사본에 그리기)
                # 중앙 타겟 박스
                cv2.rectangle(image, (cx-roi_size, cy-roi_size), (cx+roi_size, cy+roi_size), (255, 255, 255), 1)
                
                # 텍스트 표시
                cv2.putText(image, f'Max:{max_h, max_s, max_v}', (10, h-30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
                cv2.putText(image, f'Min:{min_h, min_s, min_v}', (10, h-10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                
                # 텍스트 위젯 업데이트 (가벼움)
                colorlblMax.value = f"{max_h},{max_s},{max_v}"
                colorlblMin.value = f"{min_h},{min_s},{min_v}"
        
        # 스냅샷용 이미지는 항상 최신으로 저장
        with frame_lock:
            latest_annotated_frame = image
            
        # ---------------------------------------------------------
        # [최적화 2] 화면 전송(Display) 빈도 줄이기
        # 매번 보내면 렉 걸림 -> 3번에 1번만 전송 (약 0.2초 간격)
        # ---------------------------------------------------------
        draw_counter += 1
        if draw_counter % 3 == 0:
            image_widget.value = bgr8_to_jpeg(image)
        
        # [최적화 3] CPU 양보 (주행 스레드를 위해 충분히 쉼)
        time.sleep(0.05)

# 기존 스레드 정리 후 재시작
try:
    th_flag = False # 기존 스레드 종료 신호
    time.sleep(0.5)
    # camera.unobserve_all() # 필요시 주석 해제
except:
    pass

th_flag = True
thread = threading.Thread(target=colorRecog)
thread.start()

In [3]:
# --------------------------------------------------------------
# 3. 스냅샷 버튼 기능
# --------------------------------------------------------------
SAVE_DIR = "hsv_snapshots"
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

snapshot_btn = ipywidgets.Button(description='Snapshot Info', button_style='danger', icon='camera')
snapshot_msg = ipywidgets.Label(value="준비됨")

def save_snapshot(b):
    with frame_lock:
        if latest_annotated_frame is not None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            h_val = colorlblMax.value.split(',')[0]
            filename = f"HSV_{timestamp}_H{h_val}.jpg"
            save_path = os.path.join(SAVE_DIR, filename)
            
            # 정보가 그려진 이미지 저장
            cv2.imwrite(save_path, latest_annotated_frame)
            snapshot_msg.value = f"저장됨: {filename}"
        else:
            snapshot_msg.value = "데이터 없음"

snapshot_btn.on_click(save_snapshot)

In [4]:
# --------------------------------------------------------------
# 4. 자율주행 실행 (메인 스레드 부하 없음)
# --------------------------------------------------------------
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.VBox([
    image_widget,
    ipywidgets.HBox([colorlblMax, colorlblMin]),
    ipywidgets.HBox([snapshot_btn, snapshot_msg]),
    ipywidgets.HBox([y_slider, speed_slider]),
    ipywidgets.HBox([x_slider, steering_slider])
]))

# 자율주행 로직
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    # 주행은 원본 이미지를 바로 사용 (가장 빠름)
    image = change['new']
    
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = float(x)
    y_slider.value = float(y)
    
    speed_slider.value = 0.3 # 속도 설정
    
    angle = np.arctan2(x, y)
    pid = angle * 0.2 + (angle - angle_last) * 0.5
    angle_last = angle
    
    steering_slider.value = pid
    
    left_val = max(min(speed_slider.value + steering_slider.value, 1.0), -0.9)
    right_val = max(min(speed_slider.value - steering_slider.value, 1.0), -0.9)
    
    robot.left_motor.value = float(left_val)
    robot.right_motor.value = float(right_val)

# 자율주행 연결
execute({'new': camera.value})
camera.observe(execute, names='value')

In [ ]:
import time
camera.unobserve(execute, names='value')
time.sleep(0.1) 
robot.stop()
camera.stop()